## 3.9 多层感知机的从零开始实现
我们已经从上一节里了解了多层感知机的原理。下面，我们一起来动手实现一个多层感知机。首先导入实现所需的包或模块。

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

### 3.9.1. 读取数据集
这里继续使用Fashion-MNIST数据集。我们将使用多层感知机对图像进行分类。

In [2]:
batch_size = 256
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = tf.cast(train_images,dtype=tf.float32)/255.0
test_images = tf.cast(test_images,dtype=tf.float32)/255.0
train_iter = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).batch(batch_size)
test_iter = tf.data.Dataset.from_tensor_slices((test_images,test_labels)).batch(batch_size)

### 3.9.2. 定义模型参数
我们在“softmax回归的从零开始实现”一节里已经介绍了，Fashion-MNIST数据集中图像形状为 28×28 ，类别数为10。本节中我们依然使用长度为 28×28=784 的向量表示每一张图像。因此，输入个数为784，输出个数为10。实验中，我们设超参数隐藏单元个数为256。

In [3]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = tf.Variable(tf.random.truncated_normal(shape=[num_inputs,num_hiddens],stddev=0.01,dtype=tf.float32))
b1 = tf.Variable(tf.zeros(shape=(num_hiddens),dtype=tf.float32))
W2 = tf.Variable(tf.random.truncated_normal(shape=[num_hiddens,num_outputs],stddev=0.01,dtype=tf.float32))
b2 = tf.Variable(tf.zeros(shape=(num_outputs),dtype=tf.float32))
params = [W1,b1,W2,b2]

### 3.9.3. 定义激活函数
这里我们使用基础的`maximum`函数来实现`ReLU`，而非直接调用Tensorflow的`relu`函数。

In [4]:
def relu(X):
    return tf.math.maximum(X,0)

### 3.9.4. 定义模型
同softmax回归一样，我们通过`reshape`函数将每张原始图像改成长度为`num_inputs`的向量。然后我们实现上一节中多层感知机的计算表达式。

In [5]:
def net(X):
    X = tf.reshape(X,shape=(-1,num_inputs))
    H = relu(tf.matmul(X,W1)+b1)
    return tf.nn.softmax(tf.matmul(H,W2)+b2)

### 3.9.5. 定义损失函数
为了得到更好的数值稳定性，我们直接使用tf提供的包括softmax运算和交叉熵损失计算的函数。

In [6]:
def loss_function(y_hat,y):
    y_hat = tf.cast(y_hat,dtype=tf.float32)
    y = tf.one_hot(y,depth=y_hat.shape[-1])
    return -tf.math.log(tf.boolean_mask(y_hat,y)) # 得到标签的预测概率，我们可以使用boolean_mask函数和one_hot函数。
loss = loss_function

### 3.9.6. 训练模型
训练多层感知机的步骤和“softmax回归的从零开始实现”一节中训练softmax回归的步骤没什么区别。我们用train_ch3函数，它的实现已经在“softmax回归的从零开始实现”一节里介绍过。我们在这里设超参数迭代周期数为5，学习率为0.5。

In [7]:
def accuracy(y_hat,y):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_hat,axis=1),tf.cast(y,dtype=tf.int64)),dtype=tf.float32))

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for _, (X, y) in enumerate(data_iter):
        y = tf.cast(y,dtype=tf.int64)
        acc_sum += np.sum(tf.cast(tf.argmax(net(X), axis=1), dtype=tf.int64) == y)
        n += y.shape[0]
    return acc_sum / n

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X,y in train_iter:
            with tf.GradientTape(persistent=True) as tape:
                y_hat = net(X)
                l = tf.reduce_sum(loss(y_hat,y))
            grads = tape.gradient(l,params)
            if trainer is None:
                # 如果没有传入优化器，则使用原先编写的小批量随机梯度下降
                for i, param in enumerate(params):
                    param.assign_sub(lr*grads[i]/batch_size)
            else:
                # tf.keras.optimizers.SGD 直接使用是随机梯度下降 theta(t+1) = theta(t) - learning_rate * gradient
                # 这里使用批量梯度下降，需要对梯度除以 batch_size, 对应原书代码的 trainer.step(batch_size)
                trainer.apply_gradients(zip([grad/batch_size for grad in grads], params))
            
            y = tf.cast(y,dtype=tf.float32)
            train_l_sum += l.numpy()
            train_acc_sum += tf.reduce_sum(tf.cast(tf.argmax(y_hat,axis=1)==tf.cast(y,dtype=tf.int64),dtype=tf.int64)).numpy()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'% (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

num_epochs,lr = 5,0.5
train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr)

Instructions for updating:
Use tf.identity instead.
epoch 1, loss 0.8139, train acc 0.698, test acc 0.803
epoch 2, loss 0.4888, train acc 0.819, test acc 0.841
epoch 3, loss 0.4190, train acc 0.844, test acc 0.851
epoch 4, loss 0.3872, train acc 0.856, test acc 0.857
epoch 5, loss 0.3643, train acc 0.865, test acc 0.864


### 3.9.7. 小结
- 可以通过手动定义模型及其参数来实现简单的多层感知机。
- 当多层感知机的层数较多时，本节的实现方法会显得较烦琐，如在定义模型参数的时候。